<a href="https://colab.research.google.com/github/hemanths03/CS6910_Assignment_3/blob/main/Question_8/lyrics_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! ls

drive  eval_tmp.txt  res_tmp.txt  sample_data  train_tmp.txt  train.txt


## Changing directory

In [ ]:
%cd /content/drive/MyDrive/lyrics

/content/drive/MyDrive/lyrics


#Cloning hugging face repository into the drive in above changed path

In [ ]:
! git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


#Again changing path

In [ ]:
%cd /content/drive/MyDrive/lyrics/transformers/examples/tensorflow/language-modeling

/content/drive/MyDrive/lyrics/transformers/examples/tensorflow/language-modeling


In [ ]:
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [ ]:
!pip install pyarrow --upgrade

#Installing all the requirements required to run cloned repository

In [ ]:
!pip install -r requirements.txt

#Appending start and end tags after splitting into test and validation set

In [ ]:
from sklearn.model_selection import train_test_split
path = '/content/train.txt'

with open(path, 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]
train, eval = train_test_split(dataset, train_size=0.9, random_state=2020)#seperating out validation data
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

#append the <|title|> token to the input then rejoin with <|endoftext|> and write back out to their respective file.

with open('/content/train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('/content/eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:5572
Evaluation size: 620


In [ ]:
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-enq3z9fz
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-enq3z9fz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


#For fine tuning GPT-2 we will be using Huggingface and will use the provided script run_clm.py

In [ ]:
!python run_clm.py \
--model_type gpt2-small \
--model_name_or_path distilgpt2 \
--train_file /content/train_tmp.txt \
--do_train \
--validation_file /content/eval_tmp.txt \
--do_predict \
--per_gpu_train_batch_size 128 \
--save_steps -1 \
--num_train_epochs 100 \
--fp16 \
--output_dir=/content/res_tmp.txt

Using custom data configuration default-8791943a31354c42
Extracting data files: 100% 2/2 [00:00<00:00, 647.92it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-8791943a31354c42/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.
100% 2/2 [00:00<00:00, 1074.09it/s]
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

#using the pre-trained GPT-2 tokenizer for creating our input sequence to the model.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained(config='/content/res_tmp.txt/config.json',pretrained_model_name_or_path='/content/res_tmp.txt/tf_model.h5',)

input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/res_tmp.txt/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


#Generating lyrics

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=10000,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

#Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: I love Deep Learning
	Word count for thats how deep I am at it CONSULTIVE enough to let you know in a few seconds What if somebody wasnt CEO baby look back on their resume Did he mention that his years of experience with Motown sucked him dry consoling me got my head cold How could she not live her dream Get emotional Dont give up Plan B Yes No Imma ever say YES Thats what dreams come true Do something when your heart is racing errrr TIME Outta the box The planks In this crazy hour Jump out into action Snatch some crabs Pull off lights Trullize roof Shoppe There goes another realm where niggas cant control themselves Tone It down like an Altoids toothpick Tip Her nose straightin Go deeper than Caprices Excuse ME though Its Maybach Porsche Like candy cane Absolute right here we stand Anakin piece Digital Underground Source Of Power Pssh We all grown used just about anything Media outlets trying new things Report abuse Golly more Ease welcome Home Ye are You Knew Yeezy were real life 